In [1]:
import pandas as pd
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import re

In [2]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

url = 'https://www.cricbuzz.com/cricket-series/ipl-2021/auction/completed'

req = urllib.request.Request(url, headers=hdr)
page = urllib.request.urlopen(req, context=ctx)
html = page.read()
soup = BeautifulSoup(html, 'html.parser')

tags = soup('a')

raw_data = list()
link="cb-col cb-col-100 cb-font-14 text-center"

div = soup.find_all('a', {"class" : link})
for d in div:
    raw_data.append(re.split(r'\s{2,}', d.text))
    
print(raw_data[0])

['', 'Arjun Tendulkar', 'Bowling Allrounder • India', 'SOLD', 'Base Price 20.00', 'Lakh', 'Final Price 20.00', 'Lakh', 'Sold To MI', '']


In [3]:
name = list()
speciality = list()
country = list()
status = list()
base_price = list()
final_price = list()
team = list()

for plyr in raw_data:
    x = plyr[2].split("•")
    if len(x) > 1: 
        name.append(plyr[1])
        speciality.append(x[0])
        country.append(x[1])

        bs = int(float(plyr[4].split(" ")[2]))
        if plyr[5] == "Crore": bs *= 100
        base_price.append(bs)
         
        status.append(plyr[3])
        if plyr[3] == 'SOLD':
            fp = int(float(plyr[6].split(" ")[2]))
            if plyr[7] == "Crore": fp *= 100
            final_price.append(fp)    
            team.append(plyr[8].split()[2])
        else:
            final_price.append('-')
            team.append('-')

In [4]:
df = pd.DataFrame({'Name': name,
                   'Role': speciality,
                   'Country': country,
                   'Status': status,
                   'Base_Price(Lakh)': base_price,
                   'Final_Price(Lakh)': final_price,
                   'Team': team})

In [5]:
df.head(5)

,Name,Role,Country,Status,Base_Price(Lakh),Final_Price(Lakh),Team
0,Arjun Tendulkar,Bowling Allrounder,India,SOLD,20,20,MI
1,Akash Singh,Bowler,India,SOLD,20,20,RR
2,Pawan Negi,Bowling Allrounder,India,SOLD,50,50,KKR
3,Venkatesh Iyer,Bowler,India,SOLD,20,20,KKR
4,Hanuma Vihari,Batting Allrounder,India,UNSOLD,100,-,-


In [6]:
player_link = list()

for n in range(len(name)):
    if len(name[n].split()) > 1:
        url = f'https://search.espncricinfo.com/ci/content/site/search.html?search=+{name[n].split()[0]}%20+{name[n].split()[1]};type=player'

        req = urllib.request.Request(url, headers=hdr)
        page = urllib.request.urlopen(req, context=ctx)
        html = page.read()

        soup = BeautifulSoup(html, 'html.parser')

        link="results in-players"
        div = soup.find_all('div', {"class" : link})
        if len(div) > 0:
            i=0
            tags = soup('a')
            for tag in tags:
                if 'player' in tag.get('href', None):
                    i += 1
                    if  i == 3:
                        link_to_player = tag.get('href', None)
        else:
            link_to_player = 'N/A'           
    else:
        link_to_player = 'N/A'
        
    player_link.append(link_to_player)

In [7]:
stats_cols = ['Mat', 'Inns', 'NO', 'Runs', 'HS', 'Ave','BF', 'SR', '100', '50', '4s', '6s', 'Ct', 'St', 'Balls', 'Wkts', 'BBI', 'BBM', 'Econ', '4w', '5w']
df[stats_cols] = '-'

In [8]:
base_url = 'https://www.espncricinfo.com'

found_players_name = []

for plyr_lnk in range(len(player_link)):
    if player_link[plyr_lnk] != 'N/A':
        url = base_url + player_link[plyr_lnk]

        req = urllib.request.Request(url, headers=hdr)
        page = urllib.request.urlopen(req, context=ctx)
        html = page.read()

        soup = BeautifulSoup(html, 'html.parser')

        raw_data = list()
        link='engineTable'

        tags = soup('h1')
        
        found_players_name.append(tags[0].text.strip())

        stats = []
        table = soup.find_all('table', {"class" : link})
        if table :
            for tab in table:
                stats.append(tab.text.split())


            for i,s in enumerate(stats[0]):
                if s == 'T20s':
                    batting_stat = stats[0][i+1:i+15]
                    for x, bat in zip(stats[0][:14], batting_stat):
                        df.loc[plyr_lnk,x] = bat


            for i,s in enumerate(stats[1]):
                if s == 'T20s':
                    bowling_stat = stats[1][i+3:i+13]
                    for x, bat in zip(stats[1][2:12], bowling_stat):
                        df.loc[plyr_lnk,x] = bat
    else:
        found_players_name.append("N/A")        

In [9]:
df.head(5)

,Name,Role,Country,Status,Base_Price(Lakh),Final_Price(Lakh),Team,Mat,Inns,NO,...,6s,Ct,St,Balls,Wkts,BBI,BBM,Econ,4w,5w
0,Arjun Tendulkar,Bowling Allrounder,India,SOLD,20,20,MI,2,2,1,...,0,0,0,42,2,1/33,1/33,9.57,0,0
1,Akash Singh,Bowler,India,SOLD,20,20,RR,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2,Pawan Negi,Bowling Allrounder,India,SOLD,50,50,KKR,119,83,32,...,33,40,0,1905,96,5/22,5/22,7.44,1,1
3,Venkatesh Iyer,Bowler,India,SOLD,20,20,KKR,38,32,12,...,28,17,0,465,21,2/10,2/10,6.98,0,0
4,Hanuma Vihari,Batting Allrounder,India,UNSOLD,100,-,-,74,73,8,...,31,29,2,408,22,3/21,3/21,7.35,0,0


In [10]:
df.to_csv('auction.csv')